# 09 · Temporal Analysis & Replay

Assess cross-slice generalization and replay effects using attack outputs.

In [ ]:
# Persistent Drive + run mode setup
import os
import sys
from pathlib import Path

try:
    from google.colab import drive  # type: ignore
    DRIVE_MOUNT = Path('/content/drive')
    if not DRIVE_MOUNT.exists():
        drive.mount('/content/drive')
except Exception as exc:  # pragma: no cover
    print(f'Colab drive mount skipped: {exc}')

if Path('/content/drive').exists():
    DRIVE_ROOT = Path('/content/drive/MyDrive').resolve()
else:
    DRIVE_ROOT = Path.home().resolve()

PROJECT_ROOT = DRIVE_ROOT / 'secure-llm-mia'
if not PROJECT_ROOT.exists():
    raise FileNotFoundError('Run 00_colab_setup.ipynb first to clone the repo on Drive.')

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

os.environ['SECURE_LLM_MIA_ROOT'] = str(PROJECT_ROOT)
os.chdir(PROJECT_ROOT)

from src.utils.runtime import current_run_mode

RUN_MODE = current_run_mode()
print('PROJECT_ROOT:', PROJECT_ROOT)
print('Active run mode:', RUN_MODE.name, '-', RUN_MODE.description)

DATA_ROOT = PROJECT_ROOT / 'data'
ARTIFACTS_DIR = PROJECT_ROOT / 'artifacts'
CHECKPOINT_ROOT = PROJECT_ROOT / 'checkpoints'
for path in (DATA_ROOT, ARTIFACTS_DIR, CHECKPOINT_ROOT):
    path.mkdir(parents=True, exist_ok=True)

BHC_DATA_DIR = DRIVE_ROOT / 'mimic-iv-bhc'
BHC_DATA_DIR.mkdir(parents=True, exist_ok=True)
BHC_CSV_PATH = BHC_DATA_DIR / 'mimic-iv-bhc.csv'
print('BHC CSV path:', BHC_CSV_PATH)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

REPORTS_DIR = PROJECT_ROOT / 'reports'
FIGS_DIR = REPORTS_DIR / 'figs'
TABLES_DIR = REPORTS_DIR / 'tables'
FIGS_DIR.mkdir(parents=True, exist_ok=True)
TABLES_DIR.mkdir(parents=True, exist_ok=True)

metrics_path = REPORTS_DIR / f'metrics_core_{RUN_MODE.name}.csv'
if not metrics_path.exists():
    raise FileNotFoundError('Core metrics missing. Run notebook 06 first to populate metrics_core_*.csv files.')

metrics_df = pd.read_csv(metrics_path)
if metrics_df.empty:
    raise ValueError('Metrics dataframe is empty. Ensure notebook 06 finished successfully.')

metrics_df = metrics_df.sort_values(['track', 'slice_id'])
print('Loaded metrics rows:', len(metrics_df))
tracks = metrics_df['track'].unique().tolist()
print('Replay tracks detected:', tracks)

auc_pivot = metrics_df.pivot(index='slice_id', columns='track', values='auc')
tpr_pivot = metrics_df.pivot(index='slice_id', columns='track', values='tpr_at_0.01')

auc_delta = auc_pivot.max(axis=1) - auc_pivot.min(axis=1)
tpr_delta = tpr_pivot.max(axis=1) - tpr_pivot.min(axis=1)
summary = pd.DataFrame({
    'slice_id': auc_pivot.index,
    **{f'auc_{track}': auc_pivot[track] for track in auc_pivot.columns},
    **{f'tpr_{track}': tpr_pivot[track] for track in tpr_pivot.columns},
    'auc_delta': auc_delta,
    'tpr_delta': tpr_delta,
}).sort_values('slice_id')

summary_path = TABLES_DIR / f'temporal_summary_{RUN_MODE.name}.csv'
summary.to_csv(summary_path, index=False)
print(f'Saved temporal summary table to {summary_path}')

fig, ax = plt.subplots(figsize=(6, 4))
for track in tracks:
    subset = metrics_df[metrics_df['track'] == track]
    ax.plot(subset['slice_id'], subset['auc'], marker='o', label=track)
ax.set_xlabel('Slice ID')
ax.set_ylabel('AUC')
ax.set_title('Temporal AUC by Replay Track')
ax.grid(True, linestyle='--', alpha=0.4)
ax.legend()
auc_fig_path = FIGS_DIR / f'temporal_auc_{RUN_MODE.name}.png'
fig.savefig(auc_fig_path, dpi=200, bbox_inches='tight')
plt.close(fig)
print(f'Saved AUC plot to {auc_fig_path}')

fig, ax = plt.subplots(figsize=(6, 4))
for track in tracks:
    subset = metrics_df[metrics_df['track'] == track]
    ax.plot(subset['slice_id'], subset['tpr_at_0.01'], marker='o', label=track)
ax.set_xlabel('Slice ID')
ax.set_ylabel('TPR @ 1% FPR')
ax.set_title('Temporal TPR@1%FPR by Replay Track')
ax.grid(True, linestyle='--', alpha=0.4)
ax.legend()
tpr_fig_path = FIGS_DIR / f'temporal_tpr_{RUN_MODE.name}.png'
fig.savefig(tpr_fig_path, dpi=200, bbox_inches='tight')
plt.close(fig)
print(f'Saved TPR plot to {tpr_fig_path}')


Temporal AUC/TPR summaries saved to `reports/tables` and plots written to `reports/figs`.
